In [1]:
# Parser
from os import listdir
from os.path import isfile, join

path_project_public_1 = './project_public_1/'

files_project_public_1 = [join(path_project_public_1, f) for f in listdir(path_project_public_1) if isfile(join(path_project_public_1, f))]

print(files_project_public_1)

['./project_public_1/bdbc_model.xml', './project_public_1/h264_interactions.txt', './project_public_1/h264_example_activation.txt', './project_public_1/bdbc_feature.txt', './project_public_1/h264.xml', './project_public_1/bdbc_interactions.txt', './project_public_1/h264_feature.txt']


In [49]:
bdbc_files = 'bdbc'
xml_files = '.xml'

var_map = []

final_string = ""
variables_string = ""
p_line = "p cnf "
clauses = ""
id_of_elememt = 0
current_name = ""
current_options = []
clauses_counter = 0
optional_false_elements = []
clause_clauses = []

for element in files_project_public_1:
    if bdbc_files in element and xml_files in element:
        
        # capsule from here with an path to the .xml file to generate a dimacs string 
        # (input: 'element', output: 'final_string')
        
        from lxml import etree
        xml_rules = etree.parse(open(element, 'r'))
        root = xml_rules.getroot()
        for child in root :
            for subchild in child :
                for element in subchild : 
                    if element.tag == 'name':
                        var_map.append(element.text)
        
        for el in var_map:
            variables_string += "c " + str(1+var_map.index(el)) + " " + el + "\n"
        
        for child in root :
            for subchild in child :
                for element in subchild :
                    if element.tag == 'name':
                        current_name = element.text
                    if element.tag == 'excludedOptions' :
                        for option in element :
                            current_options.append(option.text)
                    if element.tag == 'optional' :
                        if element.text != 'False' :
                            current_name = ''
                            current_options = []
                if current_name != '' :
                    optional_false_elements.append(1+var_map.index(current_name))
                    for item in current_options :
                        optional_false_elements.append(1+var_map.index(item)*-1)
                    
                    clause_clauses.append(optional_false_elements)
                    optional_false_elements = []
                    
                    optional_false_elements.append(1+var_map.index(current_name))
                    for item in current_options :
                        optional_false_elements.append(1+var_map.index(item))
                    optional_false_elements.sort()
                    
                    if(clause_clauses.count(optional_false_elements) == 0) :
                        clause_clauses.append(optional_false_elements)
                    
                    optional_false_elements=[]
                current_options = []
                                                        
        p_line += str(len(var_map))+" "+str(len(clause_clauses))+"\n"
        
        for line in clause_clauses :
            for item in line :
                clauses += str(item)+" "
            clauses += "0\n"
        
# und zack fertig .... dimacs
        final_string = variables_string+p_line+clauses
        print(final_string)
        print("\n")





c 1 HAVE_CRYPTO
c 2 HAVE_HASH
c 3 HAVE_REPLICATION
c 4 HAVE_VERIFY
c 5 HAVE_SEQUENCE
c 6 HAVE_STATISTICS
c 7 DIAGNOSTIC
c 8 PS1K
c 9 PS4K
c 10 PS8K
c 11 PS16K
c 12 PS32K
c 13 CS32MB
c 14 CS16MB
c 15 CS64MB
c 16 CS512MB
p cnf 16 11
8 -7 -8 -9 -10 0
8 9 10 11 12 0
9 -6 -10 -9 -8 0
10 -6 -7 -9 -10 0
11 -6 -7 -8 -10 0
12 -6 -7 -8 -9 0
13 -12 -13 -14 0
13 14 15 16 0
14 -11 -14 -13 0
15 -11 -12 -14 0
16 -11 -12 -13 0



